In [281]:
import random
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class RavenTrack:
    def __init__(self, spaces = 5):
        self.spaces = spaces
    def decrement_raven(self, result):
        if result == 1:
            self.spaces -= 1

In [3]:
#Single responsibility, defining the dice for the game, want this to have a variable number of sides
class OrchardDie:
    def __init__(self, sides = 6, result = None):
        self.sides = sides
        self.result = None
        
    def roll(self):
        self.result = random.randint(1, self.sides)
        return self.result
        

In [4]:
OrchardDie().roll()

5

In [115]:
class FruitInventory():
    def __init__(self, orchard_die = None, fruit_amt = 4):
        if orchard_die is None:
            orchard_die = OrchardDie()
        self.fruit_types = orchard_die.sides - 2
        self.fruit_amt = fruit_amt
        self.full_inv = {}
        for fruit in range(self.fruit_types):
            self.full_inv[fruit + 3] = fruit_amt
    def decrement_fruit(self, result):
        if result in self.full_inv.keys() and self.full_inv[result] > 0:
            self.full_inv[result] -=1
    def smallest_strat(self):
        non_zero_fruits = {k: v for k, v in self.full_inv.items() if v > 0}
        smallest_fruit = min(non_zero_fruits, key=non_zero_fruits.get)
        self.full_inv[smallest_fruit] -= 1
    def largest_strat(self):
        largest_fruit = max(self.full_inv, key=self.full_inv.get)
        self.full_inv[largest_fruit] -= 1
    def random_strat(self):
        non_zero_fruits = [k for k, v in self.full_inv.items() if v > 0]
        random_fruit = random.choice(non_zero_fruits)
        self.full_inv[random_fruit] -= 1
    def check_not_zero(self):
        return any(value != 0 for value in self.full_inv.values())

In [6]:
#Test of decrement fruit and OrchardDie
o_die = OrchardDie()
o_die_res = o_die.roll()
print(o_die_res)
apple = FruitInventory(o_die)
apple.decrement_fruit(o_die_res)
apple.full_inv

5


{3: 4, 4: 4, 5: 3, 6: 4}

In [119]:
#Test of smallest strat
apple = FruitInventory()
apple.full_inv[4] = 3
print(apple.full_inv)
apple.smallest_strat()
print(apple.full_inv)

{3: 4, 4: 3, 5: 4, 6: 4}
{3: 4, 4: 2, 5: 4, 6: 4}


In [8]:
#Test of largest strat
apple = FruitInventory()
apple.full_inv[4] = 5
print(apple.full_inv)
apple.largest_strat(2)
print(apple.full_inv)

{3: 4, 4: 5, 5: 4, 6: 4}
{3: 4, 4: 4, 5: 4, 6: 4}


In [9]:
#Test of random strat
o_die = OrchardDie()
apple = FruitInventory(o_die)
print(apple.full_inv)
apple.random_strat(2)
print(apple.full_inv)

{3: 4, 4: 4, 5: 4, 6: 4}
{3: 4, 4: 3, 5: 4, 6: 4}


In [10]:
#Test check zeros
apple = FruitInventory(fruit_amt = 0)
apple.full_inv[5] = 1
print(apple.full_inv)
print(apple.check_not_zero())
banana = FruitInventory()
print(banana.full_inv)
print(banana.check_not_zero())
strawberry = FruitInventory(fruit_amt = 0)
print(strawberry.full_inv)
print(strawberry.check_not_zero())

{3: 0, 4: 0, 5: 1, 6: 0}
True
{3: 4, 4: 4, 5: 4, 6: 4}
True
{3: 0, 4: 0, 5: 0, 6: 0}
False


In [11]:
class GameState:
    def __init__(self, orchard_die=None):
        self.orchard_die = orchard_die if orchard_die else OrchardDie()
        self.raven_track = RavenTrack()
        self.fruit_inventory = FruitInventory(self.orchard_die)
    
    def is_game_over(self):
        return self.raven_track.spaces <= 0 or not self.fruit_inventory.check_not_zero()

In [126]:
#code to play game based on strategy including end conditions
def play_with_strat(strat):
    game_state = GameState()
    while not game_state.is_game_over():
        result = game_state.orchard_die.roll()
        strategies = {
                'smallest': game_state.fruit_inventory.smallest_strat,
                'largest': game_state.fruit_inventory.largest_strat,
                'random': game_state.fruit_inventory.random_strat
                }
        if result != 2:
            game_state.raven_track.decrement_raven(result)
            game_state.fruit_inventory.decrement_fruit(result)
        else:
            if strat in strategies:
                strategies[strat]()
                
    return game_state

In [127]:
final_fruit_inventory = play_with_strat('smallest')
print(final_fruit_inventory.fruit_inventory.full_inv)
print(final_fruit_inventory.raven_track.spaces)

{3: 0, 4: 0, 5: 0, 6: 0}
1


In [128]:
#class to store multiple results
class GameResults:
    raven_end = 0
    fruit_end = 0

In [129]:
game_results = GameResults
print(game_results.raven_end)
game_results.raven_end +=1
print(game_results.raven_end)

0
1


In [176]:
#code to run strategy multiple times
def run_strat_ntimes(n_runs: int, strat):
    game_results = GameResults
    game_results.raven_end = 0
    game_results.fruit_end = 0
    for i in range(n_runs):
        game_state = play_with_strat(strat)
        if game_state.raven_track.spaces == 0:
            game_results.raven_end += 1
        else:
            game_results.fruit_end +=1
    return game_results    

In [174]:
game_res_test = run_strat_ntimes(10, 'random')
game_res_test.raven_end

1
2
3


3

In [16]:
#code to compare results between random, smallest, and largest strategies


In [231]:
smallest_results = run_strat_ntimes(10, 'smallest')

In [232]:
smallest_results.raven_end

6

In [183]:
largest_results = run_strat_ntimes(100000, 'largest')

In [184]:
largest_results.raven_end

36699

In [185]:
random_results = run_strat_ntimes(100000, 'random')

In [186]:
random_results.raven_end

40110

In [187]:
class MultIterGame:
    smallest_results = []
    largest_results = []
    random_results = []

In [242]:
def run_batches(n_runs, n_times):
    mult_iter_game = MultIterGame()
    mult_iter_game.smallest_results.clear()
    mult_iter_game.largest_results.clear()
    mult_iter_game.random_results.clear()
    for _ in range(n_runs):
        small_run = run_strat_ntimes(n_times, 'smallest')
        mult_iter_game.smallest_results.append(small_run.raven_end)
        large_run = run_strat_ntimes(n_times, 'largest')
        mult_iter_game.largest_results.append(large_run.raven_end)
        random_run = run_strat_ntimes(n_times, 'random')
        mult_iter_game.random_results.append(random_run.raven_end)
    return mult_iter_game

In [243]:
test_run_batches = run_batches(10,10)
print(test_run_batches.smallest_results)
print(test_run_batches.largest_results)
print(test_run_batches.random_results)

[6, 5, 2, 2, 7, 5, 5, 4, 5, 4]
[4, 5, 5, 5, 5, 2, 5, 2, 2, 3]
[3, 5, 6, 5, 5, 2, 7, 2, 5, 3]


In [244]:
real_run_batches = run_batches(10000, 10)

In [246]:
small_avg = np.mean(real_run_batches.smallest_results)

In [270]:
small_perc_loss = round(small_avg/10*100,2)

In [271]:
small_perc_loss

44.31

In [265]:
large_avg = np.mean(real_run_batches.largest_results)

In [272]:
large_perc_loss = round(large_avg/10*100,2)

In [273]:
large_perc_loss

36.96

In [275]:
round(small_perc_loss - large_perc_loss,2)

7.35

In [256]:
random_avg = np.mean(real_run_batches.random_results)

In [276]:
rand_perc_loss = round(random_avg/10*100,2)

In [277]:
rand_perc_loss

40.47

In [279]:
round(rand_perc_loss - large_perc_loss,2)

3.51